# An example of using etasl_simulator

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML


In [ ]:
# to change the width of the notebook, you can uncomment the following:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; } </style>"))

# Simulating the eTaSL controller using the high-level eTaSL driver

Import all the necessary classes and functions from the etasl_py.etasl python module:

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad

Create a etasl_simulator object. This is a helper class for (more) easy simulation of eTaSL specifications.
There is also a lower-level interface.

The constructor is defined as follows:

def __init__(self, 
                nWSR                  = 300,
                cputime               = 1000,
                regularization_factor = 1E-4
            ):
            
    creates an etasl_simulator object

    input parameters:
        nMWSR :  number of iterations of the underlying QP-solver
        cputime: maximum CPU time to spend on solver one time-step
        regularization_factor :  regularization to be used during the
                                 optimization.  The weight of the joint/feature
                                 variable velocities compared to the weight of the
                                 constraints.

In [ ]:
e = etasl_simulator(regularization_factor= 0.00001)

The first think you do with this simulator object is to read in some eTaSL definitions.  We have split up the definition
of the task and the definition of the robot:

In [ ]:
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")

Next, we define the **initial value** for the joint variables **and** their **prefered order** in the output of this
simulator object.  We can specify both robot joint variables and feature variables.  If variables are not specified,
their order will arbitrary (but after the specified variables) and their initial value will correspond to the initial value given in the eTaSL specification file
(usually ==0)

In [ ]:
pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint', 'f1']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,0,0,0,0])

We define some parameters for the simulation.  The number $N$ of simulation steps to execute and the sample period $dt$.

In [ ]:
N=4000
dt=0.005

We use these values to define the input of our eTaSL controller (as a numpy array). We then pass
this input to the etasl_simulator object, while also specifying the labels, such that the etasl_simulator
knows which columns correspond to which input channel.

We do this using **SetInputTable**, which has the following signature:

    def setInputTable(inp_lbl, inp, inpvel)
    
where *inp_lbl* corresponds to the labels of the columns, *inp* corresponds to the value of the input signal,
and the optional *inpvel* correspond to the **partial** derivatives towards time of the input signal. 

In [ ]:

time = np.arange(0,N)*dt
inp_lbl=['tgt_x','tgt_y','tgt_z']
inp=np.zeros((N, len(inp_lbl)))
inp[:,0] = np.sin(time)*0.15 + 0.7
inp[:,1] = time*0
inp[:,2] = time*0 + 0.3
inpvel = np.zeros((N, len(inp_lbl)))
inpvel[:,0] = np.cos(time)*0.15
inpvel[:,1] = time*0
inpvel[:,2] = time*0

e.setInputTable(inp_lbl,inp,inpvel)

During an *initialisation phase*, consistent values for the feature variables are computed.  This is done with the
*initialize* method, with the following signature:
    def **initialize**(self,initialval, initial_lbl, max_time, time_step, convergence_crit):

     Initializes the controller and performs an optimization to compute an optimal start value 
     for the feature variables.

     Performs the following tasks in this order:
        1) prepares the solver for the initialization problem
        2) initializes the state (robot/feature names, values and velocities)
        3) sets the initial value for robot/feature variables 
        4) performs an optimization to compute an optimal start value 
           for the feature variables (only taking into account the constraints with priority==0)
        5) prepares the solver for the exuction problem and solves one step
           (such that next steps are all hot-starts)

     Args:
        initialval:           value of robot- and feature variables you 
                              want to initialize (before initialization optimization)
                              (if not specified, the initial value of the specification
                               is used.)
        initial_lbl:          labels belonging the initial value array.
        initialization_time:  max. (virtual) time to use for initialization
        sample_time:          (virtual) sample time to use for the initialization
        convergence_crit:     convergence criterion used to stop the initialization early.
    Warning:
        - initializes time to 0, you can overwrite this in the initialval map.
        - robot variables remain identical as specified after this method call.
        - feature variables can be changed after this method call (if they are involved
          in the initialization optimization). 

     Returns:
        convergedval:         variable to return the converged values for joint and 
                              feature variables.     

In [ ]:
e.initialize(initial_jpos, pos_lbl)

The last step executes the simulation with $N$ steps and sample period $dt$:

In [ ]:
e.simulate(N=N,dt=dt)

# Plotting (using the Bokeh library to provide interactive plots)

Plotting using the Bokeh library (instead of matplotlib); Bokeh can also be used to generate an HTML file for interactive plots that can be used on a (static) website.

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Defining a simple function for plotting using the Bokeh library (allowing interactive plots):

In [ ]:
def plotv(TIME,VAR, VAR_LBL, plotw=450,ploth=250, ncols=2, tooltip="$x : $y"):
    p=[]
    for i in range(0,VAR.shape[1]):
        f = figure(x_axis_label='time [s]', 
                y_axis_label= VAR_LBL[i],     
                tooltips=tooltip)
        if i==0:
            fx=f.x_range
        else:
            f.x_range=fx
        f.line(TIME, VAR[:,i], line_width=2)
        p.append(f)

    pg = gridplot( p, ncols=ncols , plot_width=plotw, plot_height=ploth) 
    show(pg)

Plotting the **joint/feature positions** :

In [ ]:
plotv(e.TIME,e.POS,e.POS_LBL)

Plotting the **joint/feature velocities**:

In [ ]:
plotv(e.TIME,e.VEL,e.POS_LBL)

And plotting **the tracking errors**:

In [ ]:
plotw=450
ploth=250
ncols=2

p1 = figure(x_axis_label='time', 
            y_axis_label= "error x", tooltips="$x : $y")
p1.line(e.TIME, e.OUTP[:,0] - e.INP[:,0], line_width=2)

p2 = figure(x_axis_label='time', 
            y_axis_label= "error y", tooltips="$x : $y",
            x_range = p1.x_range)
p2.line(e.TIME, e.OUTP[:,1] - e.INP[:,1], line_width=2)

p3 = figure(x_axis_label='time', 
            y_axis_label= "error z", tooltips="$x : $y",
            x_range = p1.x_range)
p3.line(e.TIME, e.OUTP[:,2] - e.INP[:,2], line_width=2)

pg = gridplot(  [p1,p2, p3], plot_width=plotw, plot_height=ploth, ncols=ncols)    
show(pg)